In [1]:
import pickle

from extract_consecutive_screenshots import get_pickle_name, load_objects
from visualization import get_connection


ImportError: No module named 'GTAVisionExport_postprocessing'

In [2]:
conn = get_connection()
run_id = 11
cars = load_objects(run_id)
car1 = list(cars.values())[0]

NameError: name 'get_connection' is not defined